<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9d8ee527e7a1b9d8a6cdc572f348bbfcf85f848a8bc7836ba95ec21d5c4d8f0c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-26 10:22:04
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: -18.30 K (-0.13%)
Current PnL: -25.55 L (-16.5%)
CY Booked + Current PnL: -11.26 L (-7.27%)
-------------------
Total profit:  1.60 L
Total loss:  -27.15 L
-------------------
Total Booked + Current PnL: 15.72 L (12.3%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.1%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.18 L (63.73%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.46,-18.87,23.37,0.09,19108.0,-19013.0,81764.0,89.75,40.0,M-SC,2.85,253.0,-1.00,0.58,7.42,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.12,9.24,9.57,19.69,20045.0,17718.0,209460.0,-1.35,57.0,X-MC,2.46,78.0,0.88,1.49,16.67,XY25,NTT,AC
50,MASFIN,398.61,-0.97,-2.62,25.33,22.05,24169.0,-2565.0,95415.0,-15.99,52.0,H-SC,6.85,164.0,-0.11,0.68,38.69,XR,ATH,FINANCE
43,ITC,452.00,-0.44,-0.79,11.66,10.78,27801.0,-1891.0,238427.0,-40.74,52.0,X-LC,2.02,5.0,-0.07,1.69,4.41,X40,NTT,FMCG
52,NESTLEIND,1377.00,-0.41,10.60,9.76,21.40,28651.0,28127.0,293553.0,2.64,54.0,X-LC,5.58,12.0,0.98,2.08,17.94,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TITAGARH,1548.0,5.11,-15.43,72.34,45.74,148153.0,-37376.0,204801.0,12.78,70.0,H-SC,6.18,174.0,-0.25,1.45,33.20,XY24,NTT,ENGINEERING
44,JCHAC,2282.0,2.36,-35.76,55.66,-0.01,45695.0,-45709.0,82096.0,16704.35,38.0,M-SC,12.09,233.0,-1.00,0.58,5.07,OX40N,NTT,AC
66,SIS,528.0,2.18,-21.97,55.41,21.26,48005.0,-24396.0,86636.0,2046.63,61.0,H-SC,4.85,166.0,-0.51,0.62,17.05,OX40N,NTT,MISC
42,IRCTC,1269.0,1.77,-16.65,83.47,52.93,188206.0,-45030.0,225478.0,-38.24,58.0,H-MC,7.03,118.0,-0.24,1.60,4.59,XY24,BTT,TRAVEL
60,REPCOHOME,880.0,1.37,-12.73,117.50,89.82,293801.0,-36462.0,250043.0,-56.04,52.0,H-SC,5.49,160.0,-0.12,1.78,32.29,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
47,KANSAINER,340.0,-3.39,-21.90,45.29,13.47,95389.0,-59049.0,210618.0,-67.92,54.0,H-SC,6.86,158.0,-0.62,1.50,7.09,XY24,NTT,PAINTS
6,ASIANTILES,137.0,-2.26,0.94,79.98,81.67,75735.0,883.0,94693.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.38,XR,NTT,CERAMICS
37,IEX,219.0,-1.97,-7.43,60.56,48.63,112094.0,-14859.0,185095.0,-38.37,39.0,H-SC,13.41,136.0,-0.13,1.31,3.26,XR,NTT,MISC
80,VALIANTORG,838.0,-1.82,-42.85,204.73,74.15,191423.0,-70105.0,93500.0,-268.43,52.0,H-SC,21.82,149.0,-0.37,0.66,19.85,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.26,0.94,79.98,81.67,75735.0,883.0,94693.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.38,XR,NTT,CERAMICS
17,COALINDIA,484.83,0.78,-0.28,19.56,19.23,29347.0,-426.0,150035.0,16.96,76.0,L-LC,2.83,179.0,-0.01,1.07,20.93,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.13,-2.17,112.53,107.91,162537.0,-3205.0,144439.0,-20.55,54.0,M-SC,11.72,216.0,-0.02,1.03,7.70,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.69,-18.60,22.87,0.02,32496.0,-32467.0,142092.0,116.52,36.0,M-SC,10.61,234.0,-1.00,1.01,23.60,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.46,-18.87,23.37,0.09,19108.0,-19013.0,81764.0,89.75,40.0,M-SC,2.85,253.0,-1.00,0.58,7.42,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-3.39,-21.90,45.29,13.47,95389.0,-59049.0,210618.0,-67.92,54.0,H-SC,6.86,158.0,-0.62,1.50,7.09,XY24,NTT,PAINTS
66,SIS,528.00,2.18,-21.97,55.41,21.26,48005.0,-24396.0,86636.0,2046.63,61.0,H-SC,4.85,166.0,-0.51,0.62,17.05,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.26,0.94,79.98,81.67,75735.0,883.0,94693.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.38,XR,NTT,CERAMICS
84,WIPRO,420.0,0.00,9.95,56.68,72.27,105600.0,16861.0,186309.0,-5.63,67.0,M-LC,4.17,99.0,0.16,1.32,16.74,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.00,9.95,56.68,72.27,105600.0,16861.0,186309.0,-5.63,67.0,M-LC,4.17,99.0,0.16,1.32,16.74,XR,NTT,IT
6,ASIANTILES,137.00,-2.26,0.94,79.98,81.67,75735.0,883.0,94693.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.38,XR,NTT,CERAMICS
25,FINCABLES,1641.55,-0.13,-2.17,112.53,107.91,162537.0,-3205.0,144439.0,-20.55,54.0,M-SC,11.72,216.0,-0.02,1.03,7.70,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.23,-3.92,115.15,106.72,136456.0,-4833.0,118503.0,-52.34,41.0,H-SC,4.74,139.0,-0.04,0.84,18.54,AR,ATH,MISC
1,ABB,7934.00,0.18,-4.14,51.86,45.57,130052.0,-10843.0,250776.0,-38.36,55.0,H-MC,4.52,120.0,-0.08,1.78,6.71,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-1.46,-17.13,52.12,26.07,84322.0,-33436.0,161785.0,-59.72,29.0,X-MC,13.17,56.0,-0.40,1.15,0.76,X40N,ATH,IT
9,BAJAJHFL,181.50,0.14,-18.37,90.47,55.47,179659.0,-44702.0,198584.0,-27.60,30.0,X-MC,16.06,64.0,-0.25,1.41,1.75,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.15,-18.51,75.88,43.32,100708.0,-30152.0,132720.0,-30.03,30.0,X-SC,6.28,90.0,-0.30,0.94,1.90,X40N,ATH,ELECTRICAL
8,AWL,485.00,-0.08,-25.07,104.86,53.50,237084.0,-75655.0,226096.0,-63.76,31.0,X-MC,6.40,58.0,-0.32,1.61,0.42,XY24,NTT,FMCG
20,DABUR,735.00,0.07,-4.02,49.72,43.70,118376.0,-9982.0,238086.0,-11.99,33.0,X-MC,2.11,72.0,-0.08,1.69,10.42,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-0.41,-1.51,23.00,21.14,51498.0,-3435.0,223904.0,-8.42,53.0,X-MC,1.16,74.0,-0.07,1.59,26.20,XY24,NTT,PAINTS
15,CAMS,950.0,-0.71,-1.11,25.88,24.48,64454.0,-2805.0,249051.0,-81.09,46.0,X-SC,1.64,86.0,-0.04,1.77,21.77,X40N,NTT,MISC
78,UNITDSPR,1644.0,0.48,4.97,15.05,20.77,36989.0,11627.0,245771.0,-4.14,51.0,X-MC,1.88,62.0,0.31,1.75,12.23,X40N,NTT,BREWERIES
43,ITC,452.0,-0.44,-0.79,11.66,10.78,27801.0,-1891.0,238427.0,-40.74,52.0,X-LC,2.02,5.0,-0.07,1.69,4.41,X40,NTT,FMCG
20,DABUR,735.0,0.07,-4.02,49.72,43.70,118376.0,-9982.0,238086.0,-11.99,33.0,X-MC,2.11,72.0,-0.08,1.69,10.42,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.14,-27.00,124.87,64.15,216899.0,-64251.0,173700.0,-57.19,33.0,X-SC,4.52,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,0.01,-20.51,78.01,41.51,163322.0,-54005.0,209360.0,-5.42,37.0,X-MC,8.02,61.0,-0.33,1.49,0.30,XY25,ATH,FMCG
8,AWL,485.00,-0.08,-25.07,104.86,53.50,237084.0,-75655.0,226096.0,-63.76,31.0,X-MC,6.40,58.0,-0.32,1.61,0.42,XY24,NTT,FMCG
22,DIXON,18931.72,-1.46,-17.13,52.12,26.07,84322.0,-33436.0,161785.0,-59.72,29.0,X-MC,13.17,56.0,-0.40,1.15,0.76,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.22,-38.88,121.20,35.20,95320.0,-50023.0,78647.0,3.10,35.0,X-SC,17.16,92.0,-0.52,0.56,0.88,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.77,-27.34,96.59,42.84,45623.0,-17772.0,47234.0,-52.62,53.0,X-SC,33.30,83.0,-0.39,0.34,6.86,XY24,NTT,MISC
59,RELAXO,1176.00,-0.74,-47.89,192.28,52.31,145444.0,-69518.0,75642.0,-44.54,42.0,X-SC,7.22,91.0,-0.48,0.54,1.25,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.22,-38.88,121.20,35.20,95320.0,-50023.0,78647.0,3.10,35.0,X-SC,17.16,92.0,-0.52,0.56,0.88,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.24,-0.30,24.46,24.08,30958.0,-386.0,126564.0,-8.38,50.0,X-SC,6.62,89.0,-0.01,0.90,19.25,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.15,-18.51,75.88,43.32,100708.0,-30152.0,132720.0,-30.03,30.0,X-SC,6.28,90.0,-0.30,0.94,1.90,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.61,-9.30,33.09,20.71,113517.0,-35183.0,343054.0,-22.01,66.0,X-LC,3.01,1.0,-0.31,2.44,14.63,X40,ATH,IT
41,INFY,1972.00,0.03,13.27,18.52,34.24,65637.0,41523.0,354411.0,-11.70,67.0,X-LC,6.48,2.0,0.63,2.52,22.65,X40,NTT,IT
75,TMPV,600.00,-0.67,-26.87,68.16,22.98,112599.0,-60700.0,165198.0,-24.30,47.0,X-LC,6.21,3.0,-0.54,1.17,3.90,XY24,NTT,AUTO
81,VBL,671.64,0.63,-2.91,39.74,35.67,121865.0,-9187.0,306655.0,-14.52,56.0,X-LC,3.31,4.0,-0.08,2.18,10.54,X40N,ATH,FMCG
43,ITC,452.00,-0.44,-0.79,11.66,10.78,27801.0,-1891.0,238427.0,-40.74,52.0,X-LC,2.02,5.0,-0.07,1.69,4.41,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.57,-37.04,111.06,32.89,53988.0,-28594.0,48612.0,-700.55,56.0,L-MC,6.90,259.0,-0.53,0.35,33.87,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-2.26,0.94,79.98,81.67,75735.0,883.0,94693.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.38,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.97,-2.62,25.33,22.05,24169.0,-2565.0,95415.0,-15.99,52.0,H-SC,6.85,164.0,-0.11,0.68,38.69,XR,ATH,FINANCE
13,BSOFT,831.70,0.47,-18.14,84.21,50.79,96953.0,-25521.0,115132.0,5.14,69.0,H-SC,9.28,151.0,-0.26,0.82,34.05,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.47,-18.14,84.21,50.79,96953.0,-25521.0,115132.0,5.14,69.0,H-SC,9.28,151.0,-0.26,0.82,34.05,XR,ATH,IT
6,ASIANTILES,137.00,-2.26,0.94,79.98,81.67,75735.0,883.0,94693.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.38,XR,NTT,CERAMICS
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.00,-1.82,-42.85,204.73,74.15,191423.0,-70105.0,93500.0,-268.43,52.0,H-SC,21.82,149.0,-0.37,0.66,19.85,XR,NTT,CHEMICALS
17,COALINDIA,484.83,0.78,-0.28,19.56,19.23,29347.0,-426.0,150035.0,16.96,76.0,L-LC,2.83,179.0,-0.01,1.07,20.93,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.86
2,50,76.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.76
LC    24.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.85
X40      23.06
X40N     12.89
XR        9.87
XY25      9.15
AR        8.86
OX40N     7.60
SR        0.98
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.20
H-SC    23.90
X-LC    20.73
M-SC    11.52
X-SC     8.06
H-MC     4.81
L-SC     1.56
M-MC     1.40
M-LC     1.32
H-LC     1.10
L-LC     1.07
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.23,-7.51,42.08
IT,13.08,-14.23,71.67
FINANCE,9.65,-17.32,68.35
MISC,7.20,-28.17,80.60
ELECTRICAL,5.95,-11.91,52.90
PAINTS,5.82,-12.68,29.42
INSURANCE,4.72,-1.95,36.83
PHARMA,4.05,-3.00,35.42
AUTO,2.83,-31.49,77.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3140193.0,21
AR,1323435.0,10
XR,1273218.0,13
X40,974795.0,14
X40N,871326.0,9
OX40N,724299.0,10
XY25,353186.0,6
SR,281125.0,2
MH,76865.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3614041.0,25
M-SC,1435854.0,15
X-MC,1419001.0,16
X-LC,807497.0,11
X-SC,784524.0,8
H-MC,400955.0,3
L-SC,241907.0,3
M-LC,105600.0,1
H-LC,78077.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1249585.0      6
           AR         907213.0      5
M-SC       XY24       813689.0      6
H-SC       XR         775943.0      7
X-MC       X40        485391.0      7
           XY24       406958.0      3
           X40N       343285.0      4
H-SC       OX40N      323310.0      4
M-SC       OX40N      320107.0      5
X-SC       X40N       310606.0      3
X-LC       X40        308966.0      5
X-SC       XY24       293480.0      3
H-SC       SR         281125.0      2
X-LC       X40N       217435.0      2
H-MC       AR         212749.0      2
X-LC       XY24       188275.0      2
H-MC       XY24       188206.0      1
X-MC       XY25       183367.0      2
X-SC       X40        180438.0      2
M-SC       XR         176662.0      2
L-SC       XR         161025.0      2
M-SC       AR         125396.0      2
M-LC       XR         105600.0      1
X-LC       XY25        92821.0      2
L-SC       OX40N       80882.0      1
H-LC       AR          78077.0      1
H-SC       MH          76865.0      1
L-MC       XR          53988.0      1
M-MC       XY25        47651.0      1
L-LC       XY25        29347.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
